# Tavily research endpoint with polling

A research call invokes a longer running agent which can take minutes to do in depth research. Because of this we have created an endpoint where you can poll the response.


In [ ]:
%pip install -q tavily-python

In [ ]:
import getpass
import os
from tavily import TavilyClient
import time
import httpx
from IPython.display import display, Markdown

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY:\n")

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

## Make a Polling Request

- Let's begin by sending a request to start the research and then poll for the result.
- When you send a research request endpoint, you'll receive a response containing a "request_id" field.
- This request_id is used to track the progress of your research request.


In [ ]:
result = tavily_client.research(input="What are the latest developments in artificial intelligence?", model="mini") # model = "mini" | "pro" | "auto"
request_id = result["request_id"]

To check the progress or result of your research request, send a GET request to: https://api.tavily.com/research/{request_id}

The "status" field in the response will indicate the current state of your request and can be one of:

- "pending"
- "in progress"
- "completed"
- "failed"

When the status is completed you can access the report from the "content" field.


In [ ]:
url = "https://api.tavily.com/research/"
headers = {"Authorization": f"Bearer {TAVILY_API_KEY}"}

In [ ]:
research_report = ""
sources = []

# Poll every 10 seconds
while True:
    response = httpx.get(url + request_id, headers=headers)
    response_json = response.json()

    status = response_json["status"]
    if status == "completed":
        research_report = response_json["content"]
        sources = response_json["sources"]
        break

    if status == "failed":
        raise RuntimeError(f"Research failed: {response_json['error']}")

    print(f"Status: {status}... polling again in 10 seconds")
    time.sleep(10)

print("Research Complete!")
report = Markdown(research_report)
display(report)

In [ ]:
sources